In [1]:
! pip install python-igraph

In [2]:
import igraph as ig
import json

In [3]:
from urllib.request import urlopen
url = "https://raw.githubusercontent.com/plotly/datasets/master/miserables.json"
html = urlopen(url).read()
print(html[:10])

b'{"nodes":['


In [4]:
data = json.loads(html)
print (data.keys())

dict_keys(['nodes', 'links'])


### Nodes

In [62]:
print(len(data['nodes']))
data['nodes'][:10]

77


[{'name': 'Myriel', 'group': 1},
 {'name': 'Napoleon', 'group': 1},
 {'name': 'Mlle.Baptistine', 'group': 1},
 {'name': 'Mme.Magloire', 'group': 1},
 {'name': 'CountessdeLo', 'group': 1},
 {'name': 'Geborand', 'group': 1},
 {'name': 'Champtercier', 'group': 1},
 {'name': 'Cravatte', 'group': 1},
 {'name': 'Count', 'group': 1},
 {'name': 'OldMan', 'group': 1}]

In [63]:
labels=[]
group=[]
for node in data['nodes']:
    labels.append(node['name'])
    group.append(node['group'])

### Transform Links into Edges

In [61]:
print(len(data['links']))
data['links'][:10]

254


[{'source': 1, 'target': 0, 'value': 1},
 {'source': 2, 'target': 0, 'value': 8},
 {'source': 3, 'target': 0, 'value': 10},
 {'source': 3, 'target': 2, 'value': 6},
 {'source': 4, 'target': 0, 'value': 1},
 {'source': 5, 'target': 0, 'value': 1},
 {'source': 6, 'target': 0, 'value': 1},
 {'source': 7, 'target': 0, 'value': 1},
 {'source': 8, 'target': 0, 'value': 2},
 {'source': 9, 'target': 0, 'value': 1}]

In [60]:
N=len(data['nodes'])
L=len(data['links'])
Edges=[(data['links'][k]['source'], data['links'][k]['target']) for k in range(L)]
print(len(Edges))
Edges[0]

254


(1, 0)

### Coordinates

In [70]:
# build the layout coordinates
G=ig.Graph(Edges, directed=False)
layt=G.layout('kk', dim=3)
print(len(layt))
layt[-1]

77


[-2.06610685996027, 2.8065472982693485, -2.8392694589356253]

In [90]:
# how does the kk layout work?
layt[(76, 58)[1]]

[-1.585169656873305, 1.5948818235412905, -0.9479950342077572]

In [10]:
Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]

### Graphing

In [96]:
import plotly as py
import plotly.graph_objs as go

trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             color=group,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          showspikes=False,
          title=''
          )

layout = go.Layout(
         title="Network of coappearances of characters in Victor Hugo's novel<br> Les Miserables (3D visualization)",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
             
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text="Data source: <a href='http://bost.ocks.org/mike/miserables/miserables.json'>[1] miserables.json</a>",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ],    )

In [97]:
mydata=[trace1, trace2]
fig=go.Figure(data=mydata, layout=layout)

fig.show()